In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from dateutil import parser

In [2]:
df = pd.read_csv("data.csv")
print(df.size)

141984612


In [3]:
def convert_to_datetime(row):
    dt = parser.parse(row["Date"])
    return dt

def label_month(row):
    dt = row["Datetime"]
    return dt.month

def label_hour(row):
    dt = row["Datetime"]
    return dt.year

In [ ]:
# Convert Date to Month and Hour
df["Datetime"] = df.apply(lambda row : convert_to_datetime(row), axis = 1)
df["Month"] = df.apply(lambda row : label_month(row), axis = 1)
df["Hour"] = df.apply(lambda row : label_hour(row), axis = 1)

df['sin_hour'] = np.sin(2*np.pi*df["Hour"]/24)
df['cos_hour'] = np.cos(2*np.pi*df["Hour"]/24)

df['sin_month'] = np.sin(2*np.pi*(df["Month"] - 1)/12)
df['cos_month'] = np.cos(2*np.pi*(df["Month"] - 1)/12)

In [ ]:
# Drop all data not within last 5 years
df[df['Year'].map(int) < 2013]

# Drop unnecessary columns
df.drop(["ID", "Case Number", "Date", "IUCR", "Block", "Beat", "Ward", "FBI Code", "Updated On", "Location", "Year"], inplace = True, axis=1)

# Drop rows with crime type that we don't think matter
df = df[
    (df["Primary Type"] != "GAMBLING") &
    (df["Primary Type"] != "LIQUOR LAW VIOLATION") &
    (df["Primary Type"] != "PROSTITUTION") &
    (df["Primary Type"] != "NARCOTICS") &
    (df["Primary Type"] != "PUBLIC INDECENCY") 
    ]

In [ ]:
# Drop all location features except for District for preliminary model 
df.drop(['X Coordinate', 'Y Coordinate', 'Latitude', 'Longitude', 'Community Area'], inplace = True, axis=1)

In [ ]:
# No transformations: Arrest, Domestic
# One Hot Encoded (Categorical Data): Primary Type, Description, Location Description, District
one_hot_columns = pd.get_dummies(df, columns =['Primary Type', 'Description', 'Location Description', 'District'])

# Combine one hot encoded columns with the Arrest and Domestic columns to get our official crime data!
crime = pd.concat(one_hot_columns, df["Arrest"], df["Domestic"])

In [ ]:
df[:100]